# Импортирование библиотек

In [1]:
import torch
from torchvision import transforms
from torchvision.transforms import v2
from torch.utils.data import Dataset, DataLoader
import os
import cv2
import shutil
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import random

# Обозначение констант

In [3]:
IMAGES_DIR = '../cv_open_dataset/open_img'  # Путь к вашему датасету с изображениями
MASKS_DIR = '../cv_open_dataset/open_msk'  # Путь к вашему датасету с масками
OUTPUT_DIR = '../concat_dataset/mine_synt'  # Путь к выходной директории

In [4]:
image_files = [f for f in os.listdir(IMAGES_DIR) if f.endswith(('.jpg', '.png'))]
mask_files = [f for f in os.listdir(MASKS_DIR) if f.endswith(('.jpg','.png'))]
if len(image_files) != len(mask_files):
    print(len(image_files))
    print(len(mask_files))
    print("Количество изображений и масок не совпадает.")
    for mask in mask_files:
        if mask.replace(".png", ".jpg") not in image_files:
            print(mask)

# Создание класса на основе Dataset из Pytorch

Создаём класс, в котором из директорий с путём к изображениям и к их маскам выделяем пути к файлам и далее преобразовываем изображения и их маски в одинаковом формате

In [10]:
class DirtDataset(Dataset):
    def __init__(self, data_path, masks_path):
        self.data_path = data_path # Путь к директории с фото
        self.labels_path = masks_path # Путь к директории с масками
        self.data = [os.path.join(data_path,f) for f in os.listdir(self.data_path) if f.endswith(('.jpg', '.png'))]  # Путь к каждому фото
        self.labels = [os.path.join(masks_path,f) for f in os.listdir(self.labels_path) if f.endswith(('.jpg', '.png'))] # Путь к каждой маске

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Индексируем каждый путь к фото и маскам 
        img_path = self.data[idx]
        mask_path = self.labels[idx]
        # Открываем фото и маски в коде
        image = Image.open(img_path)
        mask = Image.open(mask_path)
        # Рандомно выбираем какое преобразование будет сделано
        is_horizontal, is_vertical = random.choice([0,1]), random.choice([0,1])
        # Делаем эти преобразования для маски и фото
        if is_horizontal:
            image = v2.functional.horizontal_flip(image)
            mask = v2.functional.horizontal_flip(mask)
        if is_vertical:
            image = v2.functional.vertical_flip(image)
            mask = v2.functional.vertical_flip(mask)
        return image, mask

# Создаём папки где будем хранить аугментированные данные

In [13]:
os.makedirs(os.path.join(OUTPUT_DIR,os.path.basename(IMAGES_DIR),),exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR,os.path.basename(MASKS_DIR),),exist_ok=True)

# Через цикл перебираем все фото и маски и сохраняем их в определённые папки

In [16]:
for i,[image,mask] in enumerate(DirtDataset(IMAGES_DIR,MASKS_DIR)):
    cv2.imwrite(os.path.join(OUTPUT_DIR,os.path.basename(IMAGES_DIR),'synt_' + str(i) + '.png'), np.array(image.convert('RGB'))[:, :, ::-1])
    cv2.imwrite(os.path.join(OUTPUT_DIR,os.path.basename(MASKS_DIR),'synt_' + str(i)+ '.png'), np.array(mask.convert('RGB'))[:, :, ::-1])